In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2018/start/1"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
header = soup.find('tr', attrs={'class':'colhead'})

columns = [col.get_text() for col in header.find_all('td')]

final_df = pd.DataFrame(columns=columns)
final_df

,,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA


In [4]:
for i in range(1,331,50):

    url = "https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2018/start/{}".format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    players = soup.find_all('tr', attrs={'class':re.compile('row player-10-')})
    for player in players:

        stats = [stat.get_text() for stat in player.find_all('td')]

        temp_df = pd.DataFrame(stats).transpose()
        temp_df.columns = columns

        final_df = pd.concat([final_df, temp_df], ignore_index=True)

season2018_df = final_df
season2018_df

,,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA
0,1,Mookie Betts,6,136,520,129,180,47,5,32,80,81,91,30,6,.346
1,2,J.D. Martinez,9,150,569,111,188,37,2,43,130,69,146,6,1,.330
2,3,Jeff McNeil,2,63,225,35,74,11,6,3,19,14,24,7,1,.329
3,4,Christian Yelich,7,147,574,118,187,34,7,36,110,68,135,22,4,.326
4,5,Jose Altuve,9,137,534,84,169,29,2,13,61,55,79,17,4,.316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,,Gary Sanchez,5,89,323,51,60,17,0,18,53,46,94,1,0,.186
327,328,Aaron Altherr,6,105,243,28,44,11,1,8,38,36,91,3,2,.181
328,329,Dexter Fowler,12,90,289,40,52,10,0,8,31,38,75,5,2,.180
329,330,Sandy Leon,8,89,265,30,47,12,0,5,22,15,75,1,0,.177


In [7]:
season2018_df.to_csv('scraped_data/2018_stats.csv', index=False, sep=',', encoding='utf-8')